In [1]:
from pulp import *
from pulp import LpProblem, LpVariable, LpMinimize, LpInteger, lpSum, value, LpBinary,LpStatusOptimal
import pulp
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore", message="Overwriting previously set objective.")
import utility
import docplex.mp.model
import docplex
import docplex_explainer
import mymetrics
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
import joblib
import dill
from alibi.explainers import AnchorTabular
import ast

In [8]:
np.load(f'Sonar_results/neg_explanations_0.25.npz')['neg_exp_twostep'][0], mymetrics.range_sum(np.load(f'Sonar_results/neg_explanations_0.25.npz')['neg_exp_twostep'][0])


(array([[0.        , 1.        ],
        [0.        , 1.        ],
        [0.        , 1.        ],
        [0.        , 1.        ],
        [0.        , 1.        ],
        [0.        , 0.3628057 ],
        [0.        , 1.        ],
        [0.        , 0.24983462],
        [0.        , 1.        ],
        [0.21707422, 1.        ],
        [0.3431164 , 1.        ],
        [0.45750293, 1.        ],
        [0.56240104, 1.        ],
        [0.        , 0.42559555],
        [0.        , 0.547096  ],
        [0.        , 0.717586  ],
        [0.80696301, 1.        ],
        [0.91532468, 1.        ],
        [1.        , 1.        ],
        [0.93771404, 1.        ],
        [0.90682968, 1.        ],
        [0.93344239, 1.        ],
        [0.73338985, 1.        ],
        [0.        , 0.50271489],
        [0.        , 0.39610656],
        [0.2777839 , 1.        ],
        [0.68284484, 1.        ],
        [0.86753808, 1.        ],
        [0.89985795, 1.        ],
        [0.788

In [81]:
def load_data(dataset_name):
    if dataset_name == 'Iris':
        # Load Dataset
        dataset = datasets.load_iris()
        df = pd.DataFrame(dataset.data, columns = dataset.feature_names)
        # Scale
        scaler = MinMaxScaler()
        scaler.fit(dataset.data)
        scaled_df = scaler.transform(dataset.data)
        # Check if binary targets
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns)
        targets = (utility.check_targets_0_1(np.where(dataset.target == dataset.target[0],0,1))).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Banknote':
        # Load Dataset
        df = pd.read_csv('./datasets/banknote_authentication.csv')
        # Scale
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Blood_Transfusion':
        df = pd.read_csv('./datasets/blood_transfusion.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Breast_Cancer':
        dataset = datasets.load_breast_cancer()
        df = pd.DataFrame(dataset.data, columns = dataset.feature_names)
        scaler = MinMaxScaler()
        scaler.fit(dataset.data)
        scaled_df = scaler.transform(dataset.data)
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns)
        targets = (utility.check_targets_0_1(np.where(dataset.target == dataset.target[0],0,1))).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Climate':
        df = pd.read_csv('./datasets/climate_model_simulation_crashes.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Glass':
        df = pd.read_csv('./datasets/glass.csv')
        df['target'] = df['target'].apply(lambda x: 1 if x in [1, 2, 3] else 0)
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Ionosphere':
        df = pd.read_csv('./datasets/ionosphere.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Modeling':
        df = pd.read_csv('./datasets/User_Knowledge_Modeling.csv')
        df['target'] = df['target'].apply(lambda x: 1 if x == 'Low' else 0)
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)

    elif dataset_name == 'Parkinsons':
        df = pd.read_csv('./datasets/parkinsons.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)

    elif dataset_name == 'Pima':
        df = pd.read_csv('./datasets/diabetes.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)

    elif dataset_name == 'Sonar':
        df = pd.read_csv('./datasets/sonar.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Wine':
        dataset = datasets.load_wine()
        df = pd.DataFrame(dataset.data, columns = dataset.feature_names)
        scaler = MinMaxScaler()
        scaler.fit(dataset.data)
        scaled_df = scaler.transform(dataset.data)
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns)
        targets = (utility.check_targets_0_1(np.where(dataset.target == dataset.target[0],0,1))).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)

    elif dataset_name == 'Vertebral-Column':
        dataset_name = 'Vertebral-Column'
        df = pd.read_csv('./datasets/column_2C.dat', sep=" ", names=['pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle', 'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis','target'])
        df['target']=np.where(df['target']=='AB',1,0)
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
        
    else:
        print("Incorrect dataset name")

def parse_explanation(explanation, feature_names, epsilon=1e-6):
    bounds = [[0, 1] for _ in range(len(feature_names))]
    conditions = explanation

    for condition in conditions:
        condition_no_space = condition.replace(' ', '')  # for regex matching
        # Check for double inequality
        match = re.match(r'(\d+\.?\d*)\s*(<|<=)\s*([^\s<>]+)\s*(<|<=)\s*(\d+\.?\d*)', condition_no_space)
        
        if match:
            value_1, op1, feature_token, op2, value_2 = match.groups()
            value_1 = float(value_1)
            value_2 = float(value_2)
            lower_bound = value_1 if op1 == '<=' else value_1 + epsilon
            upper_bound = value_2 if op2 == '<=' else value_2
            upper_bound = upper_bound if op2 == '<=' else upper_bound - epsilon

            for idx, feature in enumerate(feature_names):
                if feature.replace(" ", "") in feature_token:
                    bounds[idx] = [lower_bound, upper_bound]
                    break
            continue  # go to next condition

        # Fallback to single operator logic
        for idx, feature in enumerate(feature_names):
            if feature in condition:
                cond_clean = condition.replace('<=', ' LESS_EQUAL ').replace('>=', ' GREATER_EQUAL ')
                cond_clean = cond_clean.replace('<', ' < ').replace('>', ' > ')
                tokens = cond_clean.split()

                tokens = ['<=' if token == 'LESS_EQUAL' else token for token in tokens]
                tokens = ['>=' if token == 'GREATER_EQUAL' else token for token in tokens]

                operator = None
                operator_pos = None
                for i, token in enumerate(tokens):
                    if token in ['>', '>=', '<', '<=']:
                        operator = token
                        operator_pos = i
                        break

                value = None
                if operator is not None and operator_pos is not None:
                    for i in range(operator_pos + 1, len(tokens)):
                        try:
                            value = float(tokens[i])
                            break
                        except ValueError:
                            continue

                if value is not None:
                    if operator == '>':
                        bounds[idx] = [value + epsilon, 1]
                    elif operator == '>=':
                        bounds[idx] = [value, 1]
                    elif operator == '<':
                        bounds[idx] = [0, value - epsilon]
                    elif operator == '<=':
                        bounds[idx] = [0, value]
                else:
                    print(f"Could not extract numeric value from condition: '{condition}'")

    return np.array(bounds)
def calculate_coverage_with_tolerance(dataset_df, l, tolerance=1e-7):
    df_copy = dataset_df.copy()
    bounds = np.array(l) 
    for i, (lower_bound, upper_bound) in enumerate(bounds):
        column = df_copy.columns[i]
        df_copy = df_copy[
            (df_copy[column] >= (lower_bound - tolerance)) & 
            (df_copy[column] <= (upper_bound + tolerance))
        ]
    return df_copy

In [5]:
datasets_name = ['Wine']#['Iris', 'Wine', 'Vertebral-Column', 'Pima', 'Parkinsons', 'Breast_Cancer', 'Blood_Transfusion', 'Ionosphere', 'Glass', 'Climate', 'Modeling', 'Banknote', 'Sonar']
p_values = [0.25]
for dataset_name in datasets_name:
    for p in p_values:
        path_anchors = f'./Anchors_results/{dataset_name}_results.csv' 
        path_twostep = dataset_name+ f'_results/results_{p}.csv'
        path_twostep_brute = dataset_name+ f'_results/raw_metric_data_{p}.csv'
        np.random.seed(50)
        anchors_brute_results = pd.read_csv(path_anchors)
        twostep_results = pd.read_csv(path_twostep)
        twostep_results_brute = pd.read_csv(path_twostep_brute)
        times_anchors = anchors_brute_results['time'].values
        coverage_anchors = anchors_brute_results['coverage'].values
        errors_anchors = anchors_brute_results['errors'].values
        rsum_anchors = anchors_brute_results['rsum'].values
        sizes_anchors = anchors_brute_results['size'].values
        
        times_twostep = twostep_results_brute['times_twostep'].values
        coverage_twostep = twostep_results_brute['coverage_twostep'].values
        rsum_twostep = twostep_results_brute['rsum_twostep'].values
        feature_sizes_twostep = twostep_results_brute['feature_sizes_twostep'].values
        
        
        
        
        
        # Ensure all lists are NumPy arrays
        times_twostep = np.array(times_twostep)
        coverage_twostep = np.array(coverage_twostep)
        sizes_anchors = np.array(sizes_anchors)
        feature_sizes_twostep = np.array(feature_sizes_twostep)
        rsum_anchors = np.array(rsum_anchors)
        rsum_twostep = np.array(rsum_twostep)
        
        
        
        # Compute means and standard deviations
        (time_mean_anchors, time_std_anchors) = compute_mean_std(times_anchors)
        (time_mean_twostep, time_std_twostep) = compute_mean_std(times_twostep)
        
        (coverage_mean_anchors, coverage_std_anchors) = compute_mean_std(coverage_anchors)
        (coverage_mean_twostep, coverage_std_twostep) = compute_mean_std(coverage_twostep)
        
        (sizes_mean_anchors, sizes_std_anchors) = compute_mean_std(sizes_anchors)
        (sizes_mean_twostep, sizes_std_twostep) = compute_mean_std(feature_sizes_twostep)
        
        (rsum_mean_anchors, rsum_std_anchors) = compute_mean_std(rsum_anchors)
        (rsum_mean_twostep, rsum_std_twostep) = compute_mean_std(rsum_twostep)
        
        # Compute relative percentage differences (Mean & Std)
        time_mean_diff = relative_percentage_diff(time_mean_twostep, time_mean_anchors)
        coverage_mean_diff = relative_percentage_diff(coverage_mean_twostep, coverage_mean_anchors)
        
        time_std_diff = relative_percentage_diff(time_std_twostep, time_std_anchors)
        coverage_std_diff = relative_percentage_diff(coverage_std_twostep, coverage_std_anchors)
        
        sizes_mean_diff = relative_percentage_diff(sizes_mean_twostep, sizes_mean_anchors)
        sizes_std_diff = relative_percentage_diff(sizes_std_twostep, sizes_std_anchors)
        
        rsum_mean_diff = relative_percentage_diff(rsum_mean_twostep, rsum_mean_anchors)
        rsum_std_diff = relative_percentage_diff(rsum_std_twostep, rsum_std_anchors)
        
        # Compute pointwise relative differences
        time_relative_pointwise = relative_percentage_diff(times_twostep, times_anchors)
        coverage_relative_pointwise = relative_percentage_diff(coverage_twostep, coverage_anchors)
        
        sizes_relative_pointwise = relative_percentage_diff(feature_sizes_twostep, sizes_anchors)
        rsum_relative_pointwise = relative_percentage_diff(rsum_twostep, rsum_anchors)
        
        # Compute pointwise means
        time_relative_mean = np.mean(time_relative_pointwise) 
        coverage_relative_mean = np.mean(coverage_relative_pointwise)
        sizes_relative_mean = np.mean(sizes_relative_pointwise)
        rsum_relative_mean = np.mean(rsum_relative_pointwise)
        
        # Compute pointwise standard deviations
        time_relative_std = np.std(time_relative_pointwise) 
        coverage_relative_std = np.std(coverage_relative_pointwise)
        sizes_relative_std = np.std(sizes_relative_pointwise)
        rsum_relative_std = np.std(rsum_relative_pointwise)
        
        # Organize Data
        all_metrics_data = all_metrics_data = {
            'Metric': ['Time', 'Size', 'Ranges_Sum', 'Coverage'],
            'ANCHORS_MEAN': [time_mean_anchors, sizes_mean_anchors, rsum_mean_anchors, coverage_mean_anchors],
            'ANCHORS_STD': [time_std_anchors, sizes_std_anchors, rsum_std_anchors, coverage_std_anchors],
            'TWOSTEP_MEAN': [time_mean_twostep, sizes_mean_twostep, rsum_mean_twostep, coverage_mean_twostep],
            'TWOSTEP_STD': [time_std_twostep, sizes_std_twostep, rsum_std_twostep, coverage_std_twostep],
            'MEAN_DIFF_%': [time_mean_diff, sizes_mean_diff, rsum_mean_diff, coverage_mean_diff],
            'STD_DIFF_%': [time_std_diff, sizes_std_diff, rsum_std_diff, coverage_std_diff],
            'POINTWISE_MEAN_%': [time_relative_mean, sizes_relative_mean, rsum_relative_mean, coverage_relative_mean],
            'POINTWISE_STD_%': [time_relative_std, sizes_relative_std, rsum_relative_std, coverage_relative_std],
            'ANCHORS_ERROR_RATE_MEAN%': [None, None, None, np.mean(errors_anchors/coverage_anchors)*100],
            'ANCHORS_ERROR_RATE_STD%': [None, None, None, np.std(errors_anchors/coverage_anchors)*100],
            
        }
        # Display and save
        all_metrics_df = pd.DataFrame(all_metrics_data)
        display(all_metrics_df)
        print(errors_anchors.sum())
        #all_metrics_df.to_csv(f'Anchors_vs_Twostep_Results/Original_{dataset_name}_{p}_results.csv', index=False)

,Metric,ANCHORS_MEAN,ANCHORS_STD,TWOSTEP_MEAN,TWOSTEP_STD,MEAN_DIFF_%,STD_DIFF_%,POINTWISE_MEAN_%,POINTWISE_STD_%,ANCHORS_ERROR_RATE_MEAN%,ANCHORS_ERROR_RATE_STD%
0,Time,0.187663,0.057380,0.053229,0.009428,-71.635952,-83.568404,-69.427613,9.871332,NaN,NaN
1,Size,2.171642,0.414767,7.738806,1.748898,256.357388,321.657922,264.427861,93.901766,NaN,NaN
2,Ranges_Sum,11.634103,0.233232,8.348670,0.766510,-28.239681,228.646268,-28.202048,6.819880,NaN,NaN
3,Coverage,29.619403,14.220437,1.223881,0.642311,-95.867977,-95.483180,-94.068997,9.672787,0.946644,4.026229


24


In [6]:
rsum_anchors,  rsum_twostep

(array([11.829998, 11.829998, 11.229999, 11.77    , 11.829998, 11.519998,
        11.72    , 11.519998, 11.9     , 11.519998, 11.91    , 11.759997,
        11.519998, 11.519998, 11.829998, 11.569998, 11.829998, 11.72    ,
        11.519998, 11.77    , 11.519998, 11.519998, 11.519998, 11.69    ,
        11.129998, 11.66    , 11.379998, 11.519998, 11.36    , 11.769997,
        11.45    , 11.44    , 11.829998, 11.829998, 11.519998, 11.519998,
        11.829998, 12.039997, 11.129998, 11.829998, 11.519998, 11.44    ,
        11.299999, 11.46    , 11.66    , 11.66    , 11.72    , 11.829998,
        11.46    , 11.579997, 11.829998, 11.77    , 11.129998, 11.46    ,
        11.55    , 11.9     , 11.519998, 12.039997, 11.519998, 11.65    ,
        11.72    , 10.739998, 11.46    , 11.219998, 11.829998, 11.129998,
        11.909997, 11.9     , 11.829998, 11.709999, 11.9     , 11.829998,
        11.479997, 11.079998, 11.829998, 11.72    , 11.71    , 11.519998,
        11.519998, 11.9     , 11.9    

In [9]:
index = np.argmin(rsum_anchors -  rsum_twostep)
rsum_anchors[index],rsum_twostep[index]

(11.519998, 9.85755108655944)

In [172]:
anchors_explanations = pd.read_csv('Anchors_results/Wine_explanations.csv')
anchors_explanation = ast.literal_eval(anchors_explanations['Explanation'].values[index])
anchors_explanation

['proline <= 0.16', 'alcalinity_of_ash > 0.32']

In [70]:
twostep_explanations = np.load(f'{dataset_name}_results/neg_explanations_0.25.npz')['neg_exp_twostep']
twostep_explanations

array([[[0.        , 1.        ],
        [0.        , 1.        ],
        [0.        , 1.        ],
        ...,
        [0.        , 0.52845528],
        [0.60805861, 1.        ],
        [0.78245364, 1.        ]],

       [[0.        , 1.        ],
        [0.        , 1.        ],
        [0.        , 1.        ],
        ...,
        [0.        , 0.63385489],
        [0.6959707 , 1.        ],
        [0.87874465, 1.        ]],

       [[0.        , 1.        ],
        [0.        , 1.        ],
        [0.        , 1.        ],
        ...,
        [0.        , 0.510479  ],
        [0.84268678, 1.        ],
        [0.72500566, 1.        ]],

       ...,

       [[0.71052632, 1.        ],
        [0.        , 1.        ],
        [0.        , 1.        ],
        ...,
        [0.        , 0.57874823],
        [0.52747253, 1.        ],
        [0.71825963, 1.        ]],

       [[0.59363069, 1.        ],
        [0.        , 1.        ],
        [0.        , 1.        ],
        .

In [30]:
dataset_name = 'Wine'
X_test = pd.read_csv(f'{dataset_name}_results/{dataset_name}_X_test_predicted.csv')
X_test[12:13]

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
12,0.344737,0.337945,0.588235,0.536082,0.304348,0.544828,0.373418,0.396226,0.283912,0.129693,0.260163,0.772894,0.114123,1.0


In [82]:
(X_test[X_test['target']==0])[:14]

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,0.721053,0.229249,0.705882,0.335052,0.489130,0.696552,0.516878,0.490566,0.400631,0.428328,0.528455,0.608059,0.782454,0.0
1,0.807895,0.280632,0.502674,0.381443,0.380435,0.679310,0.628692,0.169811,0.621451,0.381399,0.626016,0.695971,0.878745,0.0
4,0.797368,0.278656,0.668449,0.360825,0.554348,0.558621,0.457806,0.339623,0.264984,0.321672,0.471545,0.846154,0.725392,0.0
11,0.560526,0.320158,0.700535,0.412371,0.336957,0.627586,0.611814,0.320755,0.757098,0.375427,0.447154,0.695971,0.646933,0.0
14,0.765789,0.195652,0.486631,0.350515,0.413043,0.655172,0.675105,0.358491,0.526814,0.650171,0.520325,0.670330,0.700428,0.0
15,0.686842,0.466403,0.641711,0.237113,0.500000,0.593103,0.567511,0.075472,0.394322,0.325939,0.390244,0.765568,0.404422,0.0
16,0.786842,0.185771,0.454545,0.278351,0.282609,0.575862,0.419831,0.245283,0.495268,0.291809,0.455285,0.849817,0.539943,0.0
26,0.531579,0.258893,0.994652,0.742268,0.586957,0.568966,0.493671,0.641509,0.476341,0.196246,0.528455,0.706960,0.393723,0.0
29,0.500000,0.604743,0.689840,0.412371,0.347826,0.493103,0.436709,0.226415,0.495268,0.274744,0.447154,0.824176,0.350927,0.0
32,0.736842,0.179842,0.663102,0.340206,0.260870,0.506897,0.559072,0.169811,0.593060,0.368601,0.617886,0.769231,0.703994,0.0


In [83]:
covered_df =calculate_coverage_with_tolerance(X_test,twostep_explanations[12])
covered_df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
39,0.881579,0.22332,0.545455,0.072165,0.347826,0.8,0.696203,0.301887,0.804416,0.530717,0.585366,0.6337,0.905136,0.0


In [84]:
scaler = joblib.load(f'models/{dataset_name}_scaler.pkl')

In [116]:
instance_covered = scaler.inverse_transform(covered_df.values[:, :-1])

In [153]:
scaler.inverse_transform(np.zeros(len(X_test.values[:1][0])-1).reshape(1,-1))

array([[ 11.03,   0.74,   1.36,  10.6 ,  70.  ,   0.98,   0.34,   0.13,
          0.41,   1.28,   0.48,   1.27, 278.  ]])

In [154]:
#Bounds
np.set_printoptions(suppress=True, precision=3)
lb = scaler.inverse_transform(np.zeros(len(X_test.values[:1][0])-1).reshape(1,-1))
ub = scaler.inverse_transform(np.ones(len(X_test.values[:1][0])-1).reshape(1,-1))
for i,(a,b) in enumerate(zip(lb[0],ub[0])):
    bb = [a,b]
    print(bb,instance_covered[0][i],'\n')

[11.03, 14.83] 14.38 

[0.74, 5.8] 1.8699999999999999 

[1.36, 3.2300000000000004] 2.38 

[10.6, 30.0] 12.0 

[70.0, 162.0] 102.0 

[0.98, 3.88] 3.3 

[0.34, 5.080000000000001] 3.64 

[0.13, 0.66] 0.29 

[0.4099999999999999, 3.5800000000000005] 2.96 

[1.28, 13.000000000000002] 7.5 

[0.48, 1.71] 1.2 

[1.27, 4.0] 3.0 

[278.0, 1680.0] 1547.0 



In [175]:
#Anchors
anchors_explanations = pd.read_csv('Anchors_results/Wine_explanations.csv')
anchors_explanation = ast.literal_eval(anchors_explanations['Explanation'].values[0])
print(anchors_explanation)
parsed_explanation = parse_explanation(anchors_explanation,['alcohol', 'malic_acid', 'anyotherthing', 'alcalinity_of_ash', 'magnesium',
        'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
        'proanthocyanins', 'color_intensity', 'hue',
        'od280/od315_of_diluted_wines', 'proline'])
parsed_explanation

['proline > 0.50', 'alcohol > 0.67']


array([[0.67, 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.5 , 1.  ]])

In [192]:
mymetrics.range_sum(parsed_explanation)

11.829998000000002

In [174]:
calculate_coverage_with_tolerance(X_test,parsed_explanation)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,0.721053,0.229249,0.705882,0.335052,0.489130,0.696552,0.516878,0.490566,0.400631,0.428328,0.528455,0.608059,0.782454,0.0
1,0.807895,0.280632,0.502674,0.381443,0.380435,0.679310,0.628692,0.169811,0.621451,0.381399,0.626016,0.695971,0.878745,0.0
4,0.797368,0.278656,0.668449,0.360825,0.554348,0.558621,0.457806,0.339623,0.264984,0.321672,0.471545,0.846154,0.725392,0.0
14,0.765789,0.195652,0.486631,0.350515,0.413043,0.655172,0.675105,0.358491,0.526814,0.650171,0.520325,0.670330,0.700428,0.0
16,0.786842,0.185771,0.454545,0.278351,0.282609,0.575862,0.419831,0.245283,0.495268,0.291809,0.455285,0.849817,0.539943,0.0
32,0.736842,0.179842,0.663102,0.340206,0.260870,0.506897,0.559072,0.169811,0.593060,0.368601,0.617886,0.769231,0.703994,0.0
33,0.671053,0.181818,0.534759,0.438144,0.391304,0.648276,0.601266,0.169811,0.485804,0.479522,0.495935,0.589744,0.882311,0.0
36,0.697368,0.215415,0.534759,0.340206,0.369565,0.496552,0.495781,0.547170,0.492114,0.218430,0.609756,0.586081,0.507846,0.0
39,0.881579,0.223320,0.545455,0.072165,0.347826,0.800000,0.696203,0.301887,0.804416,0.530717,0.585366,0.633700,0.905136,0.0
47,0.878947,0.239130,0.609626,0.319588,0.467391,0.989655,0.664557,0.207547,0.558360,0.556314,0.308943,0.798535,0.857347,0.0


In [189]:
#Anchors
np.set_printoptions(suppress=True, precision=3)
lb = scaler.inverse_transform(parsed_explanation.T[0].reshape(1,-1))
ub = scaler.inverse_transform(parsed_explanation.T[1].reshape(1,-1))
for i,(a,b) in enumerate(zip(lb[0],ub[0])):
    bb = [a.round(2),b.round(2)]
    print(bb,instance_covered[0][i],i+1,feature_names[i],'\n')

[13.58, 14.83] 14.38 1 alcohol 

[0.74, 5.8] 1.8699999999999999 2 malic_acid 

[1.36, 3.23] 2.38 3 ash 

[10.6, 30.0] 12.0 4 alcalinity_of_ash 

[70.0, 162.0] 102.0 5 magnesium 

[0.98, 3.88] 3.3 6 total_phenols 

[0.34, 5.08] 3.64 7 flavanoids 

[0.13, 0.66] 0.29 8 nonflavanoid_phenols 

[0.41, 3.58] 2.96 9 proanthocyanins 

[1.28, 13.0] 7.5 10 color_intensity 

[0.48, 1.71] 1.2 11 hue 

[1.27, 4.0] 3.0 12 od280/od315_of_diluted_wines 

[979.0, 1680.0] 1547.0 13 proline 



In [190]:
#Twostep
np.set_printoptions(suppress=True, precision=3)
lb = scaler.inverse_transform(twostep_explanations[12].T[0].reshape(1,-1))
ub = scaler.inverse_transform(twostep_explanations[12].T[1].reshape(1,-1))
for i,(a,b) in enumerate(zip(lb[0],ub[0])):
    bb = [a.round(2),b.round(2)]
    print(bb,instance_covered[0][i],i,feature_names[i],'\n')

[11.03, 14.83] 14.38 0 alcohol 

[0.74, 5.8] 1.8699999999999999 1 malic_acid 

[1.36, 3.23] 2.38 2 ash 

[10.6, 12.22] 12.0 3 alcalinity_of_ash 

[70.0, 162.0] 102.0 4 magnesium 

[0.98, 3.88] 3.3 5 total_phenols 

[3.61, 5.08] 3.64 6 flavanoids 

[0.13, 0.66] 0.29 7 nonflavanoid_phenols 

[0.41, 3.58] 2.96 8 proanthocyanins 

[1.28, 13.0] 7.5 9 color_intensity 

[0.48, 1.71] 1.2 10 hue 

[2.99, 4.0] 3.0 11 od280/od315_of_diluted_wines 

[1547.0, 1680.0] 1547.0 12 proline 



In [191]:
for xx,yy in zip(twostep_explanations[12],feature_names):
    print(xx,yy)

[0. 1.] alcohol
[0. 1.] malic_acid
[0. 1.] ash
[0.    0.083] alcalinity_of_ash
[0. 1.] magnesium
[0. 1.] total_phenols
[0.689 1.   ] flavanoids
[0. 1.] nonflavanoid_phenols
[0. 1.] proanthocyanins
[0. 1.] color_intensity
[0. 1.] hue
[0.632 1.   ] od280/od315_of_diluted_wines
[0.905 1.   ] proline


In [193]:
mymetrics.range_sum(twostep_explanations[12])

9.857551086559441

In [182]:
scaler.inverse_transform(covered_df.values[:,:-1].reshape(1,-1))

array([[  14.38,    1.87,    2.38,   12.  ,  102.  ,    3.3 ,    3.64,
           0.29,    2.96,    7.5 ,    1.2 ,    3.  , 1547.  ]])

In [181]:
covered_df.values[:,:-1]

array([[0.882, 0.223, 0.545, 0.072, 0.348, 0.8  , 0.696, 0.302, 0.804,
        0.531, 0.585, 0.634, 0.905]])

In [183]:
covered_df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
39,0.881579,0.22332,0.545455,0.072165,0.347826,0.8,0.696203,0.301887,0.804416,0.530717,0.585366,0.6337,0.905136,0.0
